In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon, mapping
from shapely.ops import unary_union
import folium
from folium import plugins
import branca.colormap as cm

import warnings
warnings.filterwarnings("ignore")

In [2]:
def simplify_for_web(gdf_wgs84: gpd.GeoDataFrame,
                     tol_m: float = 250,
                     min_area_m2: float = 0) -> gpd.GeoDataFrame:
    """
    Упрощает геометрии в метрах и возвращает в WGS84.
    tol_m — допуск упрощения в метрах (100–500 м обычно ок).
    min_area_m2 — минимальная площадь объектов (м²), мелочь можно отфильтровать.
    """
    g = gdf_wgs84.copy()
    g = g.to_crs(3857)               # в метры (псевдомеркатор)
    g["geometry"] = g.geometry.buffer(0)
    g["geometry"] = g.geometry.simplify(tol_m, preserve_topology=True)
    if min_area_m2 and min_area_m2 > 0:
        g["_area_m2"] = g.geometry.area
        g = g[g["_area_m2"] >= min_area_m2].drop(columns="_area_m2")
    g = g.to_crs(4326)               # обратно в WGS84 для веб‑карт
    g["geometry"] = g.geometry.buffer(0)
    g = g[~g.geometry.is_empty & g.geometry.notna()]
    return g


def print_map(data, 
              title, 
              save_df=False, 
              region='Сахалинская область', 
              GPKG_PATH='../data/municipal_districts_poly_edit.gpkg', 
              VALUE_COL='value',
              custom_step=False,
              step_start=0, step_end=100,
              color_to='#0030C0'):
    ID_COL = 'territory_id'
    DICT_XLSX_PATH = '../data/districts.xlsx'
    NAME_COL = 'municipal_district_name_short'

    assert ID_COL in data.columns and VALUE_COL in data.columns, 'Отсутствуют необходимые столбцы'

    gdf = gpd.read_file(GPKG_PATH)
    gdf = gdf[gdf['year_to'] == 9999].copy()

    # Если у GeoDataFrame не задан CRS — предполагаем WGS84
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    else:
        gdf = gdf.to_crs(epsg=4326)

    # Привести ключ к единому типу (числовой). Удалим пробелы, затем to_numeric→Int64.
    for df_ in (data, gdf):
        df_[ID_COL] = pd.to_numeric(df_[ID_COL].astype(str).str.strip(), errors='coerce').astype('Int64')

    gdf = gdf.merge(data[[ID_COL, VALUE_COL]], on=ID_COL, how='left', validate='m:1')

    gdf_web = simplify_for_web(gdf, tol_m=250, min_area_m2=0)

    try:
        dict_df = pd.read_excel(DICT_XLSX_PATH)
        dict_df[ID_COL] = pd.to_numeric(dict_df[ID_COL].astype(str).str.strip(), errors='coerce').astype('Int64')
        cols = [ID_COL]

        for c in ['region_code', 'region_name', 'municipal_district_name_short', 'municipal_district_name']:
            if c in dict_df.columns: 
                cols.append(c)

        dict_slim = dict_df[cols].drop_duplicates(subset=[ID_COL])
        gdf_web = gdf_web.merge(dict_slim, on=ID_COL, how='left')
    except Exception as e:
        print("Не удалось добавить названия из Excel:", e)

        
    gdf_region = gdf_web[gdf_web['region_name'] == region].copy()

    center = [gdf_region.geometry.unary_union.centroid.y, gdf_region.geometry.unary_union.centroid.x]
    m_region = folium.Map(location=center, zoom_start=5, tiles='CartoDB positron', control_scale=True)

    vmin, vmax = gdf_region[VALUE_COL].min(), gdf_region[VALUE_COL].max()

    if custom_step:
        vmin = step_start
        vmax = step_end

    colormap_reg = cm.LinearColormap(
        colors=["#D7D7D7", color_to],
        vmin=vmin, vmax=vmax
    ).to_step(n=3)
    colormap_reg.caption = f"{title}"


    def style_fn_reg(feat):
        val = feat['properties'].get(VALUE_COL, None)
        color = colormap_reg(val) if val is not None else '#cccccc'
        return {'fillColor': color, 'color': '#333333', 'weight': 0.7, 'fillOpacity': 0.75}


    tooltip_reg = folium.features.GeoJsonTooltip(
        fields=[NAME_COL, VALUE_COL],
        aliases=['муниципалитет', 'value'],
        localize=True
    )

    folium.GeoJson(
        data=gdf_region.to_json(),
        name=f'Регион: {region}',
        style_function=style_fn_reg,
        tooltip=tooltip_reg
    ).add_to(m_region)

    colormap_reg.add_to(m_region)
    plugins.Fullscreen().add_to(m_region)
    folium.LayerControl(collapsed=False).add_to(m_region)

    return m_region

### Население

In [3]:
population = pd.read_csv('../final_data/territory_population.csv')
population.head()

,territory_id,value
0,1953,187027
1,1954,9868
2,1955,20975
3,1956,22145
4,1957,39675


In [ ]:
print_map(population[population['territory_id'] != 195], 'Плотность населения', color_to="#129221")

### Миграция

In [3]:
relative_migration = pd.read_csv('./transform_data/relative_migration.csv')
relative_migration.head()

,Unnamed: 0,territory_id,municipal_district_name,age_group,gender,population,migration,migration_relative
0,0,1953,городской округ город Южно-Сахалинск,Всего,Женщины,107500.0,-124.0,-0.115349
1,1,1953,городской округ город Южно-Сахалинск,Всего,Мужчины,100341.0,-1212.0,-1.207881
2,2,1953,городской округ город Южно-Сахалинск,0-14,Женщины,15401.0,-6.0,-0.038959
3,3,1953,городской округ город Южно-Сахалинск,0-14,Мужчины,15862.0,-16.0,-0.100870
4,4,1953,городской округ город Южно-Сахалинск,15-24,Женщины,8257.0,-32.0,-0.387550


In [20]:
relative_migration.describe()

,Unnamed: 0,territory_id,population,migration,migration_relative
count,180.000000,180.000000,180.000000,180.000000,180.000000
mean,89.500000,1961.000000,5084.333333,-29.377778,-0.745147
std,52.105662,5.113243,11953.655584,126.651113,2.888404
min,0.000000,1953.000000,317.000000,-1212.000000,-16.057234
25%,44.750000,1956.000000,1031.750000,-30.500000,-1.591828
50%,89.500000,1961.000000,1941.500000,-10.000000,-0.424742
75%,134.250000,1966.000000,4202.250000,8.000000,0.393785
max,179.000000,1969.000000,107500.000000,184.000000,9.783631


In [28]:
print_map(relative_migration[(relative_migration['age_group'] == 'Всего') & (relative_migration['gender'] == 'Женщины')],
        'Относительная миграция (женшины)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [29]:
print_map(relative_migration[(relative_migration['age_group'] == '15-24') & (relative_migration['gender'] == 'Женщины')],
        'Относительная миграция (Женщины, 15-24 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [30]:
print_map(relative_migration[(relative_migration['age_group'] == '25-44') & (relative_migration['gender'] == 'Женщины')],
        'Относительная миграция (Женщины, 25-44 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [31]:
print_map(relative_migration[(relative_migration['age_group'] == '45-64') & (relative_migration['gender'] == 'Женщины')],
        'Относительная миграция (Женщины, 45-64 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [33]:
print_map(relative_migration[(relative_migration['age_group'] == 'Всего') & (relative_migration['gender'] == 'Мужчины')],
        'Относительная миграция (Мужчины)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [34]:
print_map(relative_migration[(relative_migration['age_group'] == '15-24') & (relative_migration['gender'] == 'Мужчины')],
        'Относительная миграция (Мужчины, 15-24 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [35]:
print_map(relative_migration[(relative_migration['age_group'] == '25-44') & (relative_migration['gender'] == 'Мужчины')],
        'Относительная миграция (Мужчины, 25-44 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

In [36]:
print_map(relative_migration[(relative_migration['age_group'] == '45-64') & (relative_migration['gender'] == 'Мужчины')],
        'Относительная миграция (Мужчины, 45-64 года)',
        VALUE_COL='migration_relative', custom_step=True, step_start=-10, step_end=10)

### Зарплаты

In [6]:
salary = pd.read_csv('./transform_data/salary.csv')

In [ ]:
salary['okved_name'].unique()

array(['Водоснабжение', 'Все отрасли',
       'Гос. управление и военн. безопасность',
       'Административная деятельность', 'Здравоохранение', 'ИТ и связь',
       'Спорт и досуг', 'Гостиницы и общепит', 'Операции с недвижимостью',
       'Научная и проф. деятельность', 'Финансы и страхование',
       'Услуги ЖКХ', 'Обрабатывающие производства', 'Образование',
       'Прочие услуги', 'Сельское хозяйство', 'Строительство', 'Торговля',
       'Транспортировка и хранение'], dtype=object)

In [7]:
print_map(salary[(salary['okved_name'] == 'Все отрасли') & (salary['year'] == 2024)],
        'Все сферы зп')

### Траты

In [3]:
consumption_final = pd.read_csv('./transform_data/consumption_final.csv')

In [4]:
consumption_final['category'].unique()

array(['Продовольствие', 'Здоровье', 'Маркетплейсы',
       'Общественное питание', 'Транспорт', 'Все категории'], dtype=object)

In [5]:
print_map(consumption_final[consumption_final['category'] == 'Все категории'],
        'Траты по всем категориям', VALUE_COL='consumption_percent')

In [6]:
print_map(consumption_final[consumption_final['category'] == 'Транспорт'],
        'Траты на транспорт', VALUE_COL='consumption_percent')

In [7]:
print_map(consumption_final[consumption_final['category'] == 'Общественное питание'],
        'Траты на общественное питание', VALUE_COL='consumption_percent')

In [8]:
print_map(consumption_final[consumption_final['category'] == 'Маркетплейсы'],
        'Траты на маркетплейсы', VALUE_COL='consumption_percent')

In [9]:
print_map(consumption_final[consumption_final['category'] == 'Здоровье'],
        'Траты на здоровье', VALUE_COL='consumption_percent')

In [10]:
print_map(consumption_final[consumption_final['category'] == 'Продовольствие'],
        'Траты на продовольствие', VALUE_COL='consumption_percent')